In [1]:
import sys
sys.path.append(str('/home/jupyter/'))
from configparser import ConfigParser

from repo.MyConfigParser import MyConfigParser

from repo.LoadData import DataLoader
from repo.BuildModel import GenerateModelFile, CreateModel
from repo.Accuracy import AccuracyCalculator, ProphetParameterSearch, PlotAccuracyCharts
from repo.FileGenerator import *

In [2]:
updated_config = ConfigParser()
updated_config.read("/home/jupyter/eu/24 Feb Update/config ups eu.ini")

getdict = MyConfigParser()
config_dict=getdict.GetDict(updated_config)
region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

training_query = config_dict["PROPHETSETTING"]["training_query"]
_input = config_dict["PROPHETSETTING"]["input"]
_output = config_dict["PROPHETSETTING"]["output"]

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]


today = datetime.date.today().strftime("%m.%d.%Y") #'03.10.2023' #
print(today)
base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = config_dict["FILEPATH"]["constraint_file_name"]

02.06.2024


In [3]:
daily_df = GetBQData(config_dict["PROPHETSETTING"]["training_query"], config_dict["PROPHETSETTING"]["input"], config_dict["PROPHETSETTING"]["output"])

df = daily_df.copy()
df = df.groupby(['region', 'market', 'account_type', 'funnel', 'ds']).sum().reset_index()

# log Transformation
# df['cost'] = np.log(df['cost'])
# df.loc[(df['cost'] < 0) | (np.isinf(df['cost'])), 'cost'] = 0

df.ds = pd.to_datetime(df.ds)

df = df[~df[['ds', 'y']].isin([np.nan, np.inf, -np.inf]).any(axis=1)]
df['Nov_Dec_2018'] = 0
df.loc[(df['ds'] <= '2018-12-31') & (df['ds'] >= '2018-11-01'), 'Nov_Dec_2018'] = 1

df['Jul_Dec_2019'] = 0
df.loc[(df['ds'] <= '2019-12-31') & (df['ds'] >= '2019-07-01'), 'Jul_Dec_2019'] = 1


df['weekday_or_weekend'] = df['ds'].apply(lambda x: 1 if x.dayofweek > 4 else 0)

# set cap and floor for log regression
df['Month of Ds'] = df['ds'].dt.month_name()

df.tail()

,region,market,account_type,funnel,ds,X,y,Nov_Dec_2018,Jul_Dec_2019,weekday_or_weekend,Month of Ds
100562,Europe,UK,Google AdWords,Generic,2024-01-31,153.5983,3.0,0,0,0,January
100563,Europe,UK,Google AdWords,Generic,2024-02-01,153.0711,5.0,0,0,0,February
100564,Europe,UK,Google AdWords,Generic,2024-02-02,240.9610,2.0,0,0,0,February
100565,Europe,UK,Google AdWords,Generic,2024-02-03,44.9525,0.0,0,0,1,February
100566,Europe,UK,Google AdWords,Generic,2024-02-04,58.2078,1.0,0,0,1,February


#### Check if the markets here are aligned with planning file
df.market.unique()

In [4]:
small_market_str = 'India|SaudiArabia|UAE|Turkey|Ireland|Sweden|Netherlands|Belgium|Spain|Czech'
%load_ext autoreload
%autoreload 2
# small_countries, large_countries = Combine(df, 'L', small_market_str)
updated_df = Combine(df, 'L', small_market_str)
updated_df.market.unique()

/home/jupyter/repo/QueryingData.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  large_countries_ori.loc[large_countries_ori[large_countries_ori['funnel'] == 'Generic'].index, 'generic_cost'] = large_countries_ori.loc[large_countries_ori['funnel'] == 'Generic', 'X']
/home/jupyter/repo/QueryingData.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  large_countries_ori.loc[large_countries_ori[large_countries_ori['funnel'] == 'Competitor'].index, 'competitor_cost'] = large_countries_ori.loc[large_countri

array(['SM', 'France', 'Germany', 'Italy', 'Poland', 'UK'], dtype=object)

In [5]:
boundary_df = pd.read_csv(boundary_df_in_path)
boundary_df.rename(columns = {'Market': 'market', 'Funnel':'funnel'}, inplace = True)

updated_df['floor'] = 0
updated_df = pd.merge(updated_df, boundary_df, on = ['market', 'funnel'])
updated_df['cap'] = updated_df.ROAS * updated_df.X / updated_df.AOV
updated_df = updated_df.fillna(0)
updated_df.loc[round(updated_df.cap,0) == 0, 'cap'] = updated_df.cap.mean()
updated_df.drop(updated_df[(updated_df.market == 'Germany') & (updated_df.funnel == 'Non_Brand') & (updated_df.ds <'2022-01-01')].index, inplace=True)
updated_df.tail()

,ds,region,market,account_type,funnel,Identifier,X,y,Nov_Dec_2018,Jul_Dec_2019,...,generic_cost,competitor_cost,generic_cost_split,competitor_cost_split,Weekend,floor,Account Type,AOV,ROAS,cap
26065,2024-01-31,Europe,UK,-,Non_Brand,L,254.0283,5.0,0.0,0.0,...,158.7483,95.28,0.624924,0.375076,0,0,-,24.36,5.37,55.998849
26066,2024-02-01,Europe,UK,-,Non_Brand,L,336.5311,12.0,0.0,0.0,...,153.0711,183.46,0.454850,0.545150,0,0,-,24.36,5.37,74.186043
26067,2024-02-02,Europe,UK,-,Non_Brand,L,330.3510,15.0,0.0,0.0,...,241.2910,89.06,0.730408,0.269592,0,0,-,24.36,5.37,72.823681
26068,2024-02-03,Europe,UK,-,Non_Brand,L,44.9525,0.0,0.0,0.0,...,44.9525,0.00,1.000000,0.000000,1,0,-,24.36,5.37,9.909480
26069,2024-02-04,Europe,UK,-,Non_Brand,L,58.2078,2.0,0.0,0.0,...,58.2078,0.00,1.000000,0.000000,1,0,-,24.36,5.37,12.831522


In [6]:
covid_df = GetCovidData('EU')
covid_df.tail()

,ds,market,new_confirmed
12878,2022-09-17,Belgium,0
12879,2022-09-17,Austria,0
12880,2022-09-17,Turkey,0
12881,2022-09-17,Italy,0
12882,2022-09-17,United Kingdom,0


In [7]:
final_df = updated_df.merge(covid_df, how = 'left', on = ['ds', 'market'])
final_df = final_df.fillna(0)

final_df.to_csv(daily_df_out_path)
final_df.head()

,ds,region,market,account_type,funnel,Identifier,X,y,Nov_Dec_2018,Jul_Dec_2019,...,competitor_cost,generic_cost_split,competitor_cost_split,Weekend,floor,Account Type,AOV,ROAS,cap,new_confirmed
0,2017-08-10,Europe,SM,-,-,S,1356.930006,226.0,0.0,0.0,...,0.0,0.5,0.5,0,0,-,76.01,28.37,506.461048,0
1,2017-08-11,Europe,SM,-,-,S,1769.019997,623.0,0.0,0.0,...,0.0,0.5,0.5,0,0,-,76.01,28.37,660.269666,0
2,2017-08-12,Europe,SM,-,-,S,0.000000,215.0,0.0,0.0,...,0.0,0.5,0.5,1,0,-,76.01,28.37,480.541526,0
3,2017-08-13,Europe,SM,-,-,S,0.000000,43.0,0.0,0.0,...,0.0,0.5,0.5,1,0,-,76.01,28.37,480.541526,0
4,2017-08-14,Europe,SM,-,-,S,2195.589999,531.0,0.0,0.0,...,0.0,0.5,0.5,0,0,-,76.01,28.37,819.482808,0


# Analytical File Generator

In [8]:
generator = AnalyticalFileGenerator(config_dict, final_df)
analytical_file_df = generator.generate()
# analytical_file_df = pd.merge(analytical_file_df, boundary_df, on = ['market', 'funnel'])
analytical_file_df.to_csv(analytical_file_path)
# generator.SaveFile(analytical_file_name, folder_name) # X is input value which should be the sum of all variable splits

In [9]:
cuts_df = pd.read_csv(cuts_df_in_path)
cuts_df['periods'] = periods
cuts_df['periods_for_acc_cal'] = 183
cuts_df['cutoff_month'] = cutoff_month

query_item_list = [query_item for query_item in [config_dict["CLIENTINFO"]["breakdown1"], \
                             config_dict["CLIENTINFO"]["breakdown2"], \
                             config_dict["CLIENTINFO"]["breakdown3"], \
                             config_dict["CLIENTINFO"]["breakdown4"]]
                             if query_item != None]

cuts_df

,market,account_type,funnel,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,seasonality_mode,input_mode,weekday_or_weekend,include_holiday,...,generic_cost_split,competitor_cost_split,new_confirmed,Nov_Dec_2018,Jul_Dec_2019,power,growth,periods,periods_for_acc_cal,cutoff_month
0,France,-,Brand,0.05,10.00,0.01,additive,multiplicative,na,na,...,na,na,na,na,na,na,linear,366.0,183,2024-01-01
1,France,-,Non_Brand,0.05,1.00,0.01,additive,additive,na,na,...,na,na,na,na,na,na,linear,366.0,183,2024-01-01
2,Germany,-,Brand,0.05,0.01,10.00,additive,multiplicative,multiplicative,na,...,na,na,na,na,na,na,linear,366.0,183,2024-01-01
3,Germany,-,Non_Brand,0.50,1.00,0.01,additive,multiplicative,additive,na,...,na,na,na,na,na,na,logistic,366.0,183,2024-01-01
4,Italy,-,Brand,0.50,10.00,1.00,multiplicative,multiplicative,multiplicative,na,...,na,na,na,na,na,na,linear,366.0,183,2024-01-01
5,Italy,-,Non_Brand,0.05,0.10,10.00,multiplicative,additive,multiplicative,na,...,na,na,na,na,na,na,linear,366.0,183,2024-01-01
6,Poland,-,Brand,0.05,0.10,1.00,multiplicative,additive,multiplicative,na,...,na,na,na,na,na,na,linear,366.0,183,2024-01-01
7,Poland,-,Non_Brand,0.10,0.01,0.01,multiplicative,multiplicative,na,na,...,na,na,na,additive,additive,na,linear,366.0,183,2024-01-01
8,SM,-,-,0.50,0.10,0.10,additive,additive,additive,na,...,na,na,na,additive,additive,na,linear,366.0,183,2024-01-01
9,UK,-,Brand,0.50,0.01,10.00,additive,additive,multiplicative,na,...,na,na,na,na,na,na,logistic,366.0,183,2024-01-01


# Accuracy Calculator

In [11]:
# %load_ext autoreload
# %autoreload 2

calculator = AccuracyCalculator()
accuracy_df = calculator.CalculateAccuracy(cuts_df, final_df, query_item_list)
# calculator.SaveFile(bucket_name, folder_name)
# accuracy_df

02:58:51 - cmdstanpy - INFO - Chain [1] start processing
02:58:51 - cmdstanpy - INFO - Chain [1] done processing
02:58:52 - cmdstanpy - INFO - Chain [1] start processing


It's overpredicted.
Monthly Error: 28.96 Period MAE: 29.459999999999997


02:58:52 - cmdstanpy - INFO - Chain [1] done processing
02:58:53 - cmdstanpy - INFO - Chain [1] start processing


It's underpredicted.
Monthly Error: 19.07 Period MAE: 28.360000000000003


02:58:53 - cmdstanpy - INFO - Chain [1] done processing
02:58:54 - cmdstanpy - INFO - Chain [1] start processing


...

It's overpredicted.
Monthly Error: 48.15 Period MAE: 47.58


02:59:04 - cmdstanpy - INFO - Chain [1] start processing
02:59:05 - cmdstanpy - INFO - Chain [1] done processing
02:59:07 - cmdstanpy - INFO - Chain [1] start processing


It's overpredicted.
Monthly Error: 25.81 Period MAE: 25.7


02:59:07 - cmdstanpy - INFO - Chain [1] done processing


It's underpredicted.
Monthly Error: 36.56 Period MAE: 30.380000000000003


In [12]:
100-accuracy_df.groupby(['market','account_type', 'funnel']).mean(numeric_only=True)

Actual      Forecast  Percentage Error  \
market  account_type funnel                                                    
France  -            Brand     -12397.857143 -16080.250000        128.960000   
                     Non_Brand   -859.571429   -587.448571         80.927143   
Germany -            Brand     -26958.000000 -29684.785714        107.655714   
                     Non_Brand  -1221.142857   -921.564286         83.917143   
Italy   -            Brand      -9810.571429 -12298.767143        126.757143   
                     Non_Brand   -890.142857   -880.005714         98.195714   
Poland  -            Brand     -11360.714286 -11578.367143        104.112857   
                     Non_Brand  -1026.142857   -965.662857         87.784286   
SM      -            -         -23658.142857 -34961.997143        148.152857   
UK      -            Brand     -16016.285714 -20158.460000        124.558571   
                     Non_Brand  -1088.571429   -727.442857         99.024286   

                                 MAPE  Forecast Period MAE  
market  account_type funnel                                 
France  -            Brand      71.04                70.54  
                     Non_Brand  80.93                71.64  
Germany -            Brand      89.22                89.92  
                     Non_Brand  71.35                77.32  
Italy   -            Brand      73.24                74.89  
                     Non_Brand  88.79                98.98  
Poland  -            Brand      89.17                98.10  
                     Non_Brand  81.01                94.63  
SM      -            -          51.85                52.42  
UK      -            Brand      74.19                74.30  
                     Non_Brand  63.44                69.62

# Parallel Param Search

In [ ]:
# %load_ext autoreload
# %autoreload 2
search_engine = ProphetParameterSearch(power_flag = False, base_path=base_path) #, linear_flag = False 
# search_engine.add_power_params()
grid_brand = search_engine.search(cuts_df[(cuts_df.index==3)|(cuts_df.index==9)|(cuts_df.index==9)], final_df)

In [ ]:
# %load_ext autoreload
# %autoreload 2
search_engine = ProphetParameterSearch(power_flag = False, base_path=base_path) #, linear_flag = False 
# search_engine.add_power_params()
grid_brand = search_engine.search(cuts_df[(cuts_df.index==8)], final_df)

In [ ]:
# PlotAccuracyCharts(accuracy_df, cuts_df, query_item_list)

# Model Generator

In [ ]:
# %load_ext autoreload
# %autoreload 
model_generator = GenerateModelFile()
model_generator.Generate(cuts_df, final_df, query_item_list, output_bucket, folder_name, base_path=base_path)

# Constraint Setting

In [15]:
bounds = cuts_df.copy()
bounds['MinWeight'] = 0.8
bounds['MaxWeight'] = 1.2
bounds

# %load_ext autoreload
# %autoreload 2
constraint_file_generator = ConstraintFileGenerator(config_dict, analytical_file_df)
constraint_file = constraint_file_generator.Generate(bounds)
constraint_file_generator.SaveFile(output_bucket, folder_name, constraint_file_name)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Data Frame file saved


# Regional pre forecast total spend

In [ ]:
final_df[(final_df.ds>'2022-12-31')&(final_df.ds<'2023-02-01')].groupby(by='region').sum(numeric_only=1)

# Exam

In [ ]:
%load_ext autoreload
%autoreload 2
for i in cuts_df.itertuples():
    if i.market == 'Poland' and i.funnel == 'Non_Brand':
        break

import pickle
from repo.Accuracy import MakeFuture
file_name = "Poland - Non_Brand.pkl"
pickle_file_path = base_path + file_name
pickle_in = open(pickle_file_path,'rb') 
# unpickled model is the model to use, training_dataset is used to get historical spend
unpickled_model, training_dataset, model_df = pickle.load(pickle_in)

analytical_file_copy = analytical_file_df.copy()
market = 'Poland'
account_type = '-'
funnel = 'Non_Brand'

spend_df_for_forecast = analytical_file_copy[(analytical_file_copy['market'] == market) & (analytical_file_copy['account_type'] == account_type) & \
                                             (analytical_file_copy['funnel'] == funnel)].copy()
spend_df_for_forecast['ds'] = pd.to_datetime(spend_df_for_forecast.ds)

future = MakeFuture(unpickled_model, training_dataset, i, 365, spend_df_for_forecast)
# print(training_dataset.ds.max(), i)
forecast = unpickled_model.predict(future)
unpickled_model.plot_components(forecast)

from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(unpickled_model, forecast)

In [ ]:
i

In [ ]:
future[future.cap<=future.floor]

In [ ]:
forecast['month'] = forecast.ds.dt.strftime('%m')

In [ ]:
forecast.groupby('month').sum(numeric_only=1)

In [ ]:
future.X